In [1]:
# import statements
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import html
from bs4 import BeautifulSoup
import requests
import time
import re

In [3]:
df = pd.read_csv("../../data/youtube_data/princess_polly_youtube_data.csv")
df.head(2)

,Title,Links,VideoLink
0,Cozy Winter clothing haul🤎 best basics! prince...,https://us.princesspolly.com\nhttps://youtu.be...,https://www.youtube.com/watch?v=aelUoWXyVRs
1,HUGE PRINCESS POLLY SPRING/SUMMER TRY-ON HAUL!...,https://us.princesspolly.com/\nhttps://us.prin...,https://www.youtube.com/watch?v=HU45aY6OYnk


In [12]:
def scrape_princess_polly(url):
    try:
        # Set Chrome options for headless mode
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        # Initialize the WebDriver
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)

        if not (driver.current_url.startswith('https://www.princesspolly.com') or driver.current_url.startswith('https://us.princesspolly.com')):
            return {'item': 'Unsupported URL', 'url': url}

        # Get the page source after interactions
        page_source = driver.page_source
        # Close the WebDriver once done to avoid multiple instances
        driver.quit()
        # Parse the page source with Beautiful Soup
        soup = BeautifulSoup(page_source, "html.parser")

        # get name of product
        title_element = soup.find("h1", class_="product__title")
        title = title_element.text.strip()
 
        description_list = soup.find("div", class_="product-details__content-inner")
        description_items = description_list.text.split('\n')
        description_items = [item for item in description_items if item.find('%') != -1]
        
        composition = (description_items[0]).split(',')[0]
        if composition.find(': ') != -1:
           composition = composition.split(': ')[1]

        pattern = r'(\d+)%\s*([\w\s-]+?(?=\s*\d+%|$))'
        fabric_percentages = re.findall(pattern, composition)

        materials = {}

        for percent, fabric in fabric_percentages:
            fabric = fabric.lower()
            fabric = fabric.replace('\xa0', '')
            materials[fabric] = percent

        final_data = {'item': title}
        final_data.update(materials)

        return final_data
        # else:
        #     return "Data element not found on the page."
    except Exception as e:
        return {'item': 'Unsupported URL', 'url': url}

In [13]:
# URL of the webpage you want to scrape
url = 'https://us.princesspolly.com/products/recycled-nylon-layla-bikini-bottoms-brown?variant=39501925318740' # 80% reclaimed nylon 20% spandex

# Call the function to scrape the website
data = scrape_princess_polly(url)
print(data)

{'item': 'Layla high cut bikini bottoms brown', 'reclaimed nylon': '80', 'spandex': '20'}


In [18]:
def scrape_and_update(row):
    links_list = row["Links"].split('\n')
    scraped_data_combined = ""

    curr = 1

    for link in links_list:
        print(str(curr) + '/' + str(len(links_list)))

        print(f"Scraping data from link: {link}")
        scraped_data = scrape_princess_polly(link)
        if scraped_data:
            scraped_data_combined += str(scraped_data) + "\n"
            print(scraped_data)
        # time.sleep(3)
        print()

        curr += 1

    return scraped_data_combined

In [19]:
df['ScrapedData'] = df.apply(scrape_and_update, axis=1)

1/14
Scraping data from link: https://us.princesspolly.com


{'item': 'Unsupported URL', 'url': 'https://us.princesspolly.com'}

2/14
Scraping data from link: https://youtu.be/6FDM52XfzSU
{'item': 'Unsupported URL', 'url': 'https://youtu.be/6FDM52XfzSU'}

3/14
Scraping data from link: https://us.princesspolly.com/products/fawcett-jeans-dark-wash-denim-petite
{'item': 'Fawcett jeans dark wash denim petite', 'cotton': '100'}

4/14
Scraping data from link: https://us.princesspolly.com/products/juelz-sweater-beige
{'item': 'Juelz sweater beige', 'polyester': '100'}

5/14
Scraping data from link: https://us.princesspolly.com/products/endman-knit-pants-beige
{'item': 'Endman knit pants beige', 'acrylic': '41'}

6/14
Scraping data from link: https://us.princesspolly.com/products/endman-zip-knit-top-beige
{'item': 'Endman zip knit top beige', 'polyester': '100'}

7/14
Scraping data from link: https://us.princesspolly.com/products/arnim-eco-nylon-long-sleeve-top-brown?variant=39666998378580
{'item': 'Arnim long sleeve top latte', 'reclaimed nylon': '86',

python(63098) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63134) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


{'item': 'Amitri lace up top orange', 'cotton': '100'}

1/10
Scraping data from link: https://us.princesspolly.com/


python(63155) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(63191) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

python(87990) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88030) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


{'item': 'Ferry zip front sweater beige', 'acrylic': '52', 'nylon': '28', 'polyester': '20'}

11/11
Scraping data from link: https://us.princesspolly.com/products/oswin-turtleneck-sweater-grey?variant=39936888569940


python(88061) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88097) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


{'item': 'Oswin turtleneck sweater grey', 'reclaimed polyester': '97', 'elastane': '3'}

1/3
Scraping data from link: https://us.princesspolly.com/


python(88140) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(88178) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [12]:
df.head()

,Title,Links,ScrapedData
0,updating my WHOLE WARDROBE for summer *princes...,http://www.princesspolly.com.au\nhttps://www.p...,"{'item': 'Barely there tie top onyx', 'polyest..."
1,ruthlessly ranking IT GIRL brands (Réalisation...,https://poshmark.com/closet/alexasunshine83\nh...,
2,unboxing my new summer wardrobe! 📦 ft. Prince...,https://www.princesspolly.com.au\nhttps://www....,"{'item': 'Gazelle denim skort light wash', 'co..."
3,Huge Summer Princess Polly HAUL + Cute Outfit ...,https://us.princesspolly.com/\nhttps://bit.ly/...,"{'item': 'Maidenwell contrast top black', 'rec..."
4,date outfit ideas / spring try-on haul (ft. pr...,https://us.princesspolly.com/\nhttps://us.prin...,{'item': 'Wyandra off the shoulder sweater whi...


In [13]:
df.to_csv('prin_polly_materials.csv', index=False)